In [25]:
from keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)


df = pd.read_csv('../csv_dateien/starcraftFinalcsvs/allRunsButOnlyTerran.csv')

conditions = [
    (df['fraction'] == "Protoss"), #1
    (df['fraction'] == "Terraner"), #2
    (df['fraction'] == "Zerg"), #3
    (df['fraction'] == "T"), #2
]

values = [1, 2, 3, 2]

df["fraction"] = np.select(conditions, values)
df=df[df["total_army"]>0]
df=df[df["fraction"]==2]
df=df[df["supply"]<=200]

dfPrepSample = df.drop(columns=["player","winner","replay_filename", "Unnamed: 0", "total_army_value","fraction"])
len(dfPrepSample)


KeyError: 'ratio'

In [14]:
df_sample = dfPrepSample.sample(frac=1, random_state=12)
dfTrainSample = df_sample.sample(frac=0.8)

dfTrainSample.head()

,total_workers,resource_mining,total_army,supply,time
3861,32,6607,13.0,54.0,335
3775,61,30745,75.0,157.0,839
5487,39,9045,17.0,70.0,413
4113,66,43708,94.0,173.0,1089
5435,67,25466,59.0,141.0,815


In [15]:
dfTestSample = df_sample.drop(dfTrainSample.index)

dfTestSample.head()

,total_workers,resource_mining,total_army,supply,time
4023,44,10282,25.0,86.0,483
5806,47,10825,32.0,94.0,538
1117,39,9124,17.0,70.0,472
5153,32,11548,21.0,70.0,411
3551,71,18941,48.0,133.0,628


In [16]:
dfTrainFeatures = dfTrainSample.copy()
dfTrainFeatures

,total_workers,resource_mining,total_army,supply,time
3861,32,6607,13.0,54.0,335
3775,61,30745,75.0,157.0,839
5487,39,9045,17.0,70.0,413
4113,66,43708,94.0,173.0,1089
5435,67,25466,59.0,141.0,815
...,...,...,...,...,...
4422,36,7366,22.0,62.0,376
2538,24,3743,9.0,30.0,315
3479,71,30156,55.0,133.0,740
648,71,36327,105.0,196.0,1019


In [17]:

dfTrainLabels = dfTrainFeatures.pop("total_army")
dfTrainFeatures = np.array(dfTrainFeatures)
dfTrainFeatures

array([[   32.,  6607.,    54.,   335.],
       [   61., 30745.,   157.,   839.],
       [   39.,  9045.,    70.,   413.],
       ...,
       [   71., 30156.,   133.,   740.],
       [   71., 36327.,   196.,  1019.],
       [   52., 15664.,   125.,   607.]])

In [18]:
# normalize = layers.Normalization()
# normalize.adapt(dfTrainFeatures)

In [19]:
sc_model = tf.keras.Sequential([layers.Dense(50, activation="relu"),layers.Dense(30, activation="relu"),layers.Dense(20, activation="relu"), layers.Dense(1)]) # letzte lösung 30/5

sc_model.compile(loss = tf.keras.losses.MeanSquaredError(),
                      optimizer = tf.keras.optimizers.Adam())

In [20]:
sc_model.fit(dfTrainFeatures, dfTrainLabels, epochs=1000)  #350

Epoch 1/1000
76/76 [==============================] - 0s 667us/step - loss: 3944.2305
Epoch 2/1000
76/76 [==============================] - 0s 700us/step - loss: 243.9985
Epoch 3/1000
76/76 [==============================] - 0s 667us/step - loss: 267.8580
Epoch 4/1000
76/76 [==============================] - 0s 686us/step - loss: 427.6177
Epoch 5/1000
76/76 [==============================] - 0s 653us/step - loss: 275.0185
Epoch 6/1000
76/76 [==============================] - 0s 647us/step - loss: 236.6288
Epoch 7/1000
76/76 [==============================] - 0s 640us/step - loss: 437.6230
Epoch 8/1000
76/76 [==============================] - 0s 630us/step - loss: 366.2668
Epoch 9/1000
76/76 [==============================] - 0s 627us/step - loss: 359.9291
Epoch 10/1000
76/76 [==============================] - 0s 620us/step - loss: 665.7567
Epoch 11/1000
76/76 [==============================] - 0s 613us/step - loss: 290.4732
Epoch 12/1000
76/76 [==============================] - 0s 633u

In [21]:
sc_model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 50)                250       
                                                                 
 dense_5 (Dense)             (None, 30)                1530      
                                                                 
 dense_6 (Dense)             (None, 20)                620       
                                                                 
 dense_7 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,421
Trainable params: 2,421
Non-trainable params: 0
_________________________________________________________________


In [22]:
dfTestFeatures = dfTestSample.copy()

dfTestLabels = dfTestFeatures.pop("total_army")
dfTestFeatures = np.array(dfTestFeatures)
dfTestLabels.head()

4023    25.0
5806    32.0
1117    17.0
5153    21.0
3551    48.0
Name: total_army, dtype: float64

In [23]:
from sklearn.metrics import r2_score


predictedValues = sc_model.predict(dfTestFeatures)
r2_score_sc = (r2_score(dfTestLabels, predictedValues))
stri = ""
if(r2_score_sc<0):
    stri = "negative" + str(r2_score_sc)
else:
    stri = str(r2_score_sc)

print(f"The R^2 value in the test set is {stri}")

19/19 [==============================] - 0s 556us/step
The R^2 value in the test set is 0.8952787342962456


In [24]:
sc_model.save('sc_ts_model.h5')
